In [1]:
# reload modules before executing user code
%load_ext autoreload
# reload all modules every time before executing the Python code
%autoreload 2

In [1]:
pip install giotto-tda umap-learn umap-learn[plot]


  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82708 sha256=587be663f14c2d10d8f26ec3bfe96a7920f1d8366b35b6ac978e8cfdc5f42b22
  Stored in directory: c:\users\nitro\appdata\local\pip\cache\wheels\4a\66\81\205d77bd58539a1d07fb8b795fcea0b17f4d59912c0f4202b7
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=d2e5f32f24597570cba7a29cc91f7a3c8625fecc8f493c6a25cbeaba9dd29025
  Stored in directory: c:\users\nitro\appdata\local\pip\cache\wheels\c0\f0\1d\20aa0dcee54ef40c179a4a4f57fad413437864a67757013a99
  Created wheel for datashape: filename=datashape-0.5.2-py3-none-any.whl size=59453 sha256=4ee351377e9b5852c1a199d8e38ead79068069b27218e89603dfbe73b3162d5e
  Stored in directory: c:\users\nitro\appdata\local\pip\cache\wheels\42\ef\d7\781cf80d4146d76b3d2ed2510113d78c2643c842cc6c22918d
Successfully built umap-learn pynndescent datashape


In [3]:
import numpy as np
import pandas as pd
import gtda.diagrams as diag
from gtda.diagrams import Scaler, Filtering, PersistenceEntropy, BettiCurve, PairwiseDistance
from gtda.homology import VietorisRipsPersistence
import gtda.graphs as gr
from gtda.pipeline import Pipeline
from gtda.plotting import plot_point_cloud, plot_heatmap
from gtda.graphs import KNeighborsGraph, GraphGeodesicDistance
from gtda.mapper import (
    CubicalCover,
    OneDimensionalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph)
from gtda.mapper import Eccentricity, Entropy

import umap.umap_ as umap

from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff


## Builiding data

### PubChem Search

*References: Wang Y, Xiao J, Suzek TO, Zhang J, Wang J, Bryant SH. PubChem: a public information system for analyzing bioactivities of small molecules. Nucleic Acids Res. 2009 Jul;37(Web Server issue):W623-33. doi: 10.1093/nar/gkp456. Epub 2009 Jun 4. PMID: 19498078; PMCID: PMC2703903.*

In [4]:
def load_csv(path):
    return pd.read_csv(path, 
                       engine='c', 
                       parse_dates=True, 
                       infer_datetime_format=True, 
                       low_memory=False)

DF = load_csv('DDH Data with Properties.csv')
DF = DF.dropna()

In [5]:
def min_max(df):
    min_max_scaler = MinMaxScaler(feature_range=(0,1))
    df_scaled = min_max_scaler.fit_transform(df[[item for item in df.columns]])
    df_scaled= pd.DataFrame(df_scaled)
    df = df.mask(df==0).fillna(method='backfill')
    return df

In [6]:
df = DF.drop(columns=['CID', 	'SMILES', 	'MolecularFormula', 'InChI', 	'InChIKey', 	'IUPACName'])

In [7]:
""" pIC50 = 50% Log inhibitory concentration of the compound (in microM).
Higher values of pIC50 indicate exponentially more potent inhibitors"""

#train values
clean = df[df['pIC50'] != 'BLINDED']
clean['pIC50'] = clean['pIC50'].astype('float')
clean = clean.dropna()
clean = clean.reset_index(drop=True)

#test values
predict = df[df['pIC50'] == 'BLINDED'] 

### Target feature (pIC50)

In [8]:
fig = px.histogram(clean, x='pIC50', nbins=15, 
                   histnorm='probability density',
                   color_discrete_sequence=['indianred'])
fig.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')

In [9]:
dfmm = pd.DataFrame(MinMaxScaler().fit_transform(clean))
dfmm.columns = clean.columns
fig = px.imshow(dfmm)
fig.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')
fig.show()

# Topological Data Analysis

<center><BIG>  
Algebra is generous; she often gives more than is asked for</BIG>
</center>
<p>
<small> Jean d’Alembert (1717-1783)</small>

Topological Data Analysis (TDA) is now an emerging area for analyzing complex data. TDA refers to a class of methods that collect information from topological structures in topological space's data. The reasons behind this rising where described by Carlsson in his article *'Topology and data'*  writen in 2009 $\rightarrow$ ( https://www.ams.org/journals/bull/2009-46-02/S0273-0979-09-01249-X/S0273-0979-09-01249-X.pdf ) :<p>
    $\circledcirc$ <u>Qualitative information is needed</u> to obtain knowledge about the data <p>
    $\circledcirc$ <u>Metrics are not theoretically justified</u>. Apart from physical systems where the phenomena studied often support  clean explanatory theories which  tell one exactly what metric to use, in other kind of systems is not clear how much significance has actual distances. <p>
    $\circledcirc$ <u>Coordinates are not natural</u> Instead of  restrict ourselves  to  studying  properties of  thedata which depend on any particular choice of coordinate, we should look for meaning out of this coordinates cause often they hasn't. <p>
        $\circledcirc$ <u>Summaries are more valuable than individual parameter choices.</u> Since real things don’t depend on how you choose to describe them, the parts of a theory corresponding to real things should be invariants. It is therefore productive to develop other mechanisms in which the behavior of invariants can be effectively summarized. <p>

TDA combines algebraic topology and other tools from pure mathematics to allow a useful study of shape of the data. The most widely discussed topologies of data include connected components, tunnels, voids, etc., of a topological space. 

## Manifold Learning and topology

### Uniform Manifold Approximation and Projection  for Dimensionality Reduction

UMAP is a new technique by McInnes et al. that offers a number of advantages over t-SNE, most notably increased speed and better preservation of the data's global structure (https://umap-learn.readthedocs.io/en/latest/how_umap_works.html).

UMAP, at its core, works very similarly to t-SNE - both use graph layout algorithms to arrange data in low-dimensional space. In the simplest sense, UMAP constructs a high dimensional graph representation of the data, then optimizes a low-dimensional graph to be as structurally similar as possible. Maths behind UMAP are complex but the intuition behind is remarkably simple.

UMAP is a stochastic algorithm – it makes use of randomness both to speed up approximation steps, and to aid in solving hard optimization problems. 

### Simplicical Complex

*The effect of connecting points as  increasing some radius arund them results in the creation of geometric objects called simplices.*

Simplicical complexes provide a particularly simple combinatorial way to describe certain topological spaces. A simplicical complex structure on a space is an expression of the space as <u> a union of points, intervals, triangles, and higher dimensional analogues</u>.<p> It can be defined as a pair $( 𝑉 , Σ )$, where $𝑉$ is a finite set, and $Σ $is a family of non-empty subsets of $𝑉$ such that $σ ∈ Σ$ ($σ$ is a i-simplex) and   $τ ⊆ σ$ implies that $τ ∈ Σ$. Associated to a simplicial complex is a topological space $|( 𝑉 , Σ )|$. The building blocks of a simplicial complex are called simplices (plural of simplex). For example a 0-simplex is just a point, a 1-simplex is two points connected with a line segment, a 2-simplex is a filled triangle. <p>
For a given subset $Σ𝑘$, the number of elements of $σ$, is equal to $𝑘+1$ (we can write it as $(σ)=𝑘+1$).

### Fuzzy simplicial complex 

In order to construct the initial high-dimensional graph, UMAP builds something called **fuzzy simplicial complex**. A FSC is a representation of a weighted graph, with edge weights representing the likelihood that two points are connected.

To determine connectedness, UMAP extends a radius outwards from each point, connecting points when those radii overlap. Choosing this radius is critical - too small a choice will lead to small, isolated clusters, while too large a choice will connect everything together. UMAP overcomes this challenge by choosing a radius locally, based on the distance to each point's nth nearest neighbor. UMAP then makes the graph "fuzzy", by decreasing the likelihood of connection as the radius grows. Finally, by stipulating that each point must be connected to at least its closest neighbor, UMAP ensures that local structure is preserved in balance with global structure.

In [10]:
""" Uniform Manifold Approximation and Projection 
for Dimensionality Reduction -UMAP"""
features = clean.loc[:, :'pIC50']
#UMAP class
n_neighbors=10
min_dist=0.5 
"""default 0.1 (smaller values will 
result in a more clustered/clumped embedding)"""

umap_2d = umap.UMAP(n_neighbors=n_neighbors,
                    #metric=metric,
                    n_components=2,
                    min_dist=min_dist,
                    init='random', 
                    random_state=0) #n_components = dimensions
umap_3d = umap.UMAP(n_neighbors=n_neighbors,
                    #metric=metric, 
                    n_components=3,#n-components = dimensions
                    min_dist=min_dist,
                    init='random',
                    random_state=0) 

proj_2d = umap_2d.fit_transform(clean.drop(columns='pIC50'))
proj_3d = umap_3d.fit_transform(clean.drop(columns='pIC50'))

fig_2d = px.scatter(
    proj_2d, x=0, y=1,
    color=clean['pIC50'], labels={'color': 'pIC50'}
)
fig_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
    color=clean['pIC50'], labels={'color': 'pIC50'}
)
fig_2d.update_layout(title='UMAP projection 2D and 3D')
fig_3d.update_traces(marker_size=5)
fig_2d.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')
fig_3d.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')
fig_2d.show()
fig_3d.show()

In [11]:
#set label for classifications
label = pd.cut(clean['pIC50'], 
               bins=[-3.0,  0.0, 1.5],
               labels=['Non effective','Effective'])

fig_2d = px.scatter(
    proj_2d, x=0, y=1,
    color=label, labels={'color': 'pIC50'}
)
fig_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
    color=label, labels={'color': 'pIC50'}
)
fig_2d.update_layout(title='UMAP projection 2D and 3D')
fig_3d.update_traces(marker_size=5)
fig_2d.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')
fig_3d.update_layout({'plot_bgcolor': 'aliceblue' , #or azure
'paper_bgcolor': 'white',}, template='plotly_white')
fig_2d.show()
fig_3d.show()

## Data visualization with Point Clouds

In [12]:
plot_point_cloud(proj_3d)

### Mapper algorithm

`Mapper` is a method forthe qualitative analysis, simplification and visualization of high dimensional data sets, as well as the qualitative analysisof functions on these data sets. <u>`Mapper` can be used to generate a topological graph that captures the salient features of the data</u>.<p>
The fundamental idea behind `Mapper` is  partial clustering, it is to apply standard clustering algorithms to subsets of the original data set, and then to understand the interaction of the partial clusters formed in this way with each other, where these intersections are used in building a simplicial complex.<p>
`Mapper` does't  obtain a fully accurate representationof a data set, but rather a low-dimensional image which is easy to understand, and which can point to areas of interest[1].<p>
<small>[1] Eurographics Symposium on Point-Based Graphics (2007)M. Botsch, R. Pajarola (Editors) *'Topological Methods for the Analysis of High Dimensional Data Sets and 3D Object'*, Singh, Mémoli & Carlsson, https://research.math.osu.edu/tgda/mapperPBG.pdf</small>

In [13]:
""" 1. Define filter function – can be any scikit-learn transformer
 It is returning a selection of columns of the data """
#filter_func = Projection() #columns=[0,1]
#filter_func = Entropy() 
filter_func = Eccentricity(metric= 'euclidean') #Eccentricities of points in a point cloud or abstract metric space.
""" The distance function (metric) can be: ‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, 
‘correlation’, ‘cosine’, ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘jensenshannon’, 
‘kulsinski’, ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, 
‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’."""
""" 2. Define cover """
cover = CubicalCover(n_intervals=30,
                     overlap_frac=0.3)
""" 3. Choose clustering algorithm – default is DBSCAN """
clusterer = DBSCAN(eps=8, 
                   min_samples=3, 
                   metric='euclidean')

""" 4. Initialise pipeline """
pipe_mapper = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=-1)

In [14]:
data = clean.drop(columns='pIC50')
#Check the cluster performance
from sklearn import metrics
db = clusterer.fit(data)
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
#print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#print("Adjusted Rand Index: %0.3f"
#print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#print("Adjusted Mutual Information: %0.3f"
#     % metrics.adjusted_rand_score(labels_true, labels))
#      % metrics.adjusted_mutual_info_score(labels_true, labels))
"""The best value of Silhouette score is 1, and the worst value is -1. Values near 0 indicate overlapping clusters.
 Negative values generally indicate that a sample has been assigned to the wrong cluster, 
 as a different cluster is more similar."""
print("Silhouette Coefficient: %0.3f"
    % metrics.silhouette_score(proj_3d, labels))

Estimated number of clusters: 1
Estimated number of noise points: 88
Silhouette Coefficient: 0.003


In [15]:
plotly_params = {"node_trace": {"marker_colorscale": "RdBu"}}
fig = plot_static_mapper_graph(pipe_mapper,
                               data, 
                               layout='fruchterman_reingold',
                               color_by_columns_dropdown=True,
                               color_variable =clean['pIC50'],
                               node_scale =20,
                               plotly_params=plotly_params)
fig.show(config={'scrollZoom': True})

Figure shows captured the salient topological features of our underlying data. In our case, we haven't any hole, just a series of connected clusters. We can see how a 'circle' is drawn, but points are not closer enough to be connected. <p>
Each Nodes of the Mapper graph goups from one to several points form the time series and are colored by the mean value of the points that belong to a given node. Color encodes density values are estimated using a Gaussian Kernel as follows:<p>
<center> 
$f_ε(x) = C_ε \displaystyle\sum_{y} exp \Bigg(\frac{-d(x,y)^2}{ε}\Bigg)$ <p>
where: $(x,y) ∈ X$,<p>
$ε > 0$,<p>
$C_ε$ is a constant.<center>

In [16]:
#['pullback_set_label', 'partial_cluster_label', 'node_elements']
graph = pipe_mapper.fit_transform(data)
node_elements = graph.vs["node_elements"]
partial_cluster_label = graph.vs["partial_cluster_label"]
pullback_set_label=graph.vs["pullback_set_label"]
ne = pd.DataFrame([node_elements]).T.rename(columns={0:'Node elements'})
pcl = pd.DataFrame([partial_cluster_label]).T.rename(columns={0:'Partial Clusters Label'})
psl= pd.DataFrame([pullback_set_label]).T.rename(columns={0:'Pullback Set Label'})
NE = ne.T

In [26]:
pd.set_option('max_colwidth', 400)
DF.T[NE[23][0]].T['SMILES']

66                  ClC1=CC(OC(=O)C2=CC3=C(OC2=O)C=CC=C3)=CN=C1
67                 CN1CCN(CC1)S(=O)(=O)C1=CC2=C(NC(=O)C2=O)C=C1
71    O=C(N1CCN(CC1)S(=O)(=O)C1=CC2=C(NC(=O)C2=O)C=C1)C1=CC=CO1
Name: SMILES, dtype: object

### Structural properties of mapper graph

In [ ]:
print(f'Graph Summary:{graph.summary(verbosity=1)}')
print(f'Graph Diameter = {graph.diameter()}')

In [ ]:
print("Number of vertices in the graph:", graph.vcount())
print("Number of edges in the graph", graph.ecount())
print("Is the graph directed:", graph.is_directed())
print("Maximum degree in the graph:", graph.maxdegree())
print("Alpha Index:", graph.alpha())
#Alpha Index is a measure of connectivity which evaluates the number of cycles 
#in a graph in comparison with the maximum number of cycles.

In [ ]:
#page rank: numerical weighting to each element 
pr = pd.DataFrame(graph.vs.pagerank())
te = ne.merge(pr, left_index=True, right_index=True)
te = te.rename(columns={0 : 'Page rank'})
te.sort_values(by='Page rank', ascending=False)

In [ ]:
# graph communities
gc = pd.DataFrame(graph.community_infomap())
print(f'There are {len(gc)} communities in graph:')
gc = gc.fillna('', inplace=False)
gc

In [ ]:
print(graph.path_length_hist())

## Introduction to Persistent homology and Point clouds

<u>Persistent homology</u> refers to a class of methods for measuring topological features of shapes and functions. It converts the data into <u>simplicial complexes</u> and describes the <u>topological structure of a space</u> at different spatial resolutions. Topologies that are more persistent are detected over a wide range of spatial scales and are deemed more likely to represent true features of the underlying space rather than sampling variations, noise, etc. The input data for the computation of persistent homology is represented as a <u>point cloud</u>, a collection of data points (objects or space) defined by a given coordinates system.
    
    
<p><h4>Definitions:</h4><p> 
    Informally, clustering refers to the process of partitioning a set of data intoa number of parts or clusters. In the context of finite metric spaces, this means roughly that points withinthe clusters are nearer to each other than they are to points in different clusters. Clustering should be thought of as the statistical counterpart to the geometric construction of the path-connected components of a space, which is the fundamental building block upon which algebraic topology is based.<p>  
<u>Clustering time series</u> Suppose that we have data $X$ which varies with time. One could then ask for information concerning the behavior of clusterings produced by clusterings over time. Clusters can appear, vanish, merge, or split into separate clusters. The analysis of this behavior can be studied using <u>functoriality</u>. For $t_0<t_1$, we let $X[t_0,t_1]$ denote the set of points in the data set occurring between times $t_0$ and $t_1$. If we have $t_0<t_1<t_2<t_3$, then we have a diagram of <u>point cloud data sets.</u><p>
        
<u>Point Clouds</u>. Point clouds are finite sets of points equipped with a distance function. They are finite samples taken from a geometric object.<p> 

<u>Simplicial complex</u>. Already introduced.<p>
        
<u> Filtration</u>. A filtration on a simplicial complex $Σ$ is a collection of subcomplexes ${Σ(t)|t ∈ R}$ of $Σ$ such that $Σ(t)⊂K(t')$ whenever $t ≤ t'$. The filtration value of a simplex $σ∈K$ is the smallest $t$ such that $σ∈Σ(t)$. A filtered simplicial complex is a simplicial complex equipped with a filtration. Filtration will tune how data points are converted into the simplicial complexes and thus, how converting set points into a topological space.

### Persistence diagrams

Families of spaces parametrized by a single parameter $\epsilon$ are a way of extracting <u>connectivity information</u> from a point cloud or finite metric space. <p>
<u>Vietoris-Rips filtration</u>. Let $X$ denote a metric space, with metric $d$.  Then the Vietoris-Rips complex for $X$ (a simplicial complex), attached to the parameter $\epsilon$ (threshold value). It is denoted by $VR(X,\epsilon)$, so it will be the simplicial complex whose vertex set is $X$, and where ${x_0,x_1,...,x_k}$ spans a $k$-simplex if and only if $d(x_i,x_j) ≤ \epsilon $ for all $0 ≤ i, j ≤ k $. $VR(X,\epsilon)$ encodes useful information about the topology of the underlying metric space.<p>


In [ ]:
plot_point_cloud(proj_3d)

In [ ]:
from gtda.homology import VietorisRipsPersistence

""" Connectivity information
0-dimensional homology β0 or H0, measures clusters; 
1-dimensional homology1 β1 or H1, measures loops; and
2- dimensional homology β2 or H2, measures voids (empty spaces) """
homology_dimensions = [0, 1, 2]
VR = VietorisRipsPersistence(
                        homology_dimensions=homology_dimensions,
                        coeff=3,
                        n_jobs=-1)

diagram =VR.fit_transform(np.array(proj_3d)[None, : , :])

#plot persistence diagram
VR.fit_transform_plot(np.array(proj_3d).reshape(1, *np.array(proj_3d).shape))

In [ ]:
#scale the persistence diagram
diagramScaler = Scaler()
scaled = diagramScaler.fit_transform(diagram)
diagramScaler.plot(scaled, sample=0)

In [ ]:
from gtda.diagrams import PersistenceImage, PersistenceLandscape
# Persistence image (raster image of persistence diagram)
PI = PersistenceImage(sigma=0.5)
PI.fit_transform_plot(scaled)

### Persistence Entropy

<u>Persistent entropy (PE)</u> is a measure of the graph complexity, acting as a global topological feature for topological classification. PE or $H$ is a <u>Shannon-like entropy for the measurement of the information discovered during the filtration process</u> within the TDA. The maximum persistent entropy corresponds to the situation in which all the intervals in the barcode are of equal length or $H = log n$, being $n$ the number of input collections of persistence diagrams. Or, maxim entropy according Shannon correspond to a uniform distribution. The value of the $H$ decreases as more intervals of different length are present, setting away from an uniform distribution.

In [ ]:
persistence_entropy = PersistenceEntropy()

# calculate topological feature matrix
feat_matrix = persistence_entropy.fit_transform(diagram)

# expect shape - (n_point_clouds, n_homology_dims)
feat_matrix.shape

In [ ]:
#Persistence Enrtropy
PE0 = feat_matrix[:, 0][0] #PE for dimension 0
PE1 = feat_matrix[:, 1][0] #PE for dimension 1
PE2 = feat_matrix[:, 2][0] #PE for dimension 2
# Maxim PE, corresponding to a continuous distribution
MaxPE0 =  len(diagram[:,:,2][0][diagram[:,:,2][0]==0]) # Max PE for dimension 0
MaxPE1 = len(diagram[:,:,2][0][diagram[:,:,2][0]==1])  # Max PE for dimension 1
MaxPE2 = len(diagram[:,:,2][0][diagram[:,:,2][0]==2]) # Max PE for dimension 2

print(f'Persistence Entropy PE-H0={PE0:.2f},  PE-H1={PE1:.2f},  PE-H2={PE2:.2f}')
print(f'Max Persistence Entropy H0={MaxPE0:.2f}, Max PE H1={MaxPE1:.2f}, Max PE H2={MaxPE2:.2f}')

We have found a way to represent each point cloud in terms of just three numbers

### Persistent Betti Numbers and Betti Curves

<u> Barcodes </u>. Barcodes are a finite set of pairs $(m, n)$, where $m$ and $n$ are a non-negative integers that are used to viusalize how filtration topological characteristics are changing while $\epsilon$ changes.<p>

    
<u>Betti numbers</u>. Informally, the $k_{th}$ Betti number refers to the number of $k$-dimensional holes on a topological surface. We can describe a topological space by its own <u>topological invariants</u>, called Betti numbers or $β$. In order to derive topological signatures from a complex, we select a threshold  $\epsilon$ as a fraction of the covering radius of the point cloud, and we determine the Betti numbers $β_0,β_1$, and $β_2$ of the witness complex with this given threshhold value of $\epsilon$. For example $β_0$ is the number of connected components, $β_1$ represents 2- dimensional holes (for eg. an empty circle), $β_2$ the voids in a 3-dimensional space (for eg. a torus), etc... For instance, a $k$- dimensional empty sphere has all Betti numbers equal to zero except for $β_0$  = $β_k$ = $1$.<p>


In [ ]:
#Plot a sample from a collection of Betti curves
BC = BettiCurve()
y_betti_curves = BC.fit_transform(diagram)
BC.plot(y_betti_curves)

It can be seen that dataset has three local β1 bars from around 1.4 Å to 2.4 Å. But they differ greatly in the global region, where A-T pair contributes one significant β1 bar from around 2.9 Å to 3.6 Å, while G-C pair generates two. These barcode fingerprints characterize the intrinsic DNA structure properties, i.e., local and global loop/ring motifs

## Quantitative analysis: train a classifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    clean.drop(columns=['pIC50']), np.array(clean['pIC50']) , test_size=0.25, random_state=666)

### Binary classifier

In [ ]:
# binarize y_train, y_test for creating labels
label_train = pd.cut(y_train, 
               bins=[-3.0,  0.0, 1.5],
               labels=[0, 1])

label_test = pd.cut(y_test, 
               bins=[-3.0,  0.0, 1.5],
               labels=[0, 1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from gtda.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from gtda.point_clouds import ConsistentRescaling, ConsecutiveRescaling

#PIPELINE 1
steps = [
  
  ("persistence", VietorisRipsPersistence(metric="euclidean", 
                                          homology_dimensions=[0, 1, 2], 
                                          n_jobs=-1)),
  ("entropy", PersistenceEntropy()),
  ("model", RandomForestClassifier(n_estimators=500)
  ),
]
pipeline1 = Pipeline(steps, verbose=False)

In [ ]:
pipeline1["model"].fit(np.array(X_train),
                       np.array(label_train))

y_pred1 = pipeline1["model"].predict(np.array(X_test))
test_mse1 = accuracy_score(np.array(label_test),
                           y_pred1)
print(f'Classification Accuracy Score = {test_mse1*100:.1f} %')

###Regressor 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#PIPELINE 2
steps2 = [
    ("persistence", VietorisRipsPersistence(metric="euclidean", homology_dimensions=homology_dimensions, n_jobs=6)),
    ("entropy", PersistenceEntropy()),
    ("model", RandomForestRegressor(n_estimators=2000),
    ),
]
pipeline2 = Pipeline(steps2)

In [ ]:
from sklearn.metrics import precision_score, mean_squared_error, r2_score
pipeline2["model"].fit(np.array(X_train),
                       np.array(y_train))
y_pred2 = pipeline2["model"].predict(np.array(X_test))
test_mse2 = mean_squared_error(np.array(y_test),
                               y_pred2)
print(f'Regression Mean Squared Error = {test_mse2:.3f}')

### Standard classifier's accuracy comparison

In [ ]:
rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train,label_train)
y_std_pred = rf.predict(X_test)
test_mse_std = mean_squared_error(label_test, y_std_pred)
print(f'Classification Accuracy Score = {test_mse_std*100:.1f} %')

## Predict blinded pIC50

In [ ]:
y_pred_blinded = pipeline2["model"].predict(np.array(predict.drop(columns=['pIC50'])))
y_pred_blinded

In [ ]:
y_pred_blinded_2 = rf.predict(np.array(predict.drop(columns=['pIC50'])))
y_pred_blinded_2